<a href="https://colab.research.google.com/github/vn322/Synergy_DBM_301_CS/blob/main/05_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

In [5]:
# !/usr/local/opt/python@3.9/bin/python3.9 -m pip install nltk

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [8]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [9]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [10]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]

In [11]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [12]:
len(words) # всего слов

173403

In [13]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [14]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [15]:
len(stopwords_ru)

151

In [16]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                       for sent in sents_tokenize ]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [17]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=cf6b454dbcd3dd02c04d1422ea7ee14ba5a1a4d207976c4b7edc4021dddb76b9
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авеньон и пленил пап!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [19]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

In [20]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета. 

In [21]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [22]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [23]:
len(words) # всего слов

93069

In [24]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [25]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [26]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [27]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import gensim
gensim.__version__

'4.3.1'

In [29]:
#%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

(0, 0)

In [30]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [31]:
len(model.wv.key_to_index)

4768

In [32]:
'старуха' in model.wv.key_to_index

True

## 3. Свойства модели

In [33]:
# вектор слова
model.wv['старуха'][:10]

array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
        0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ],
      dtype=float32)

In [34]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [35]:
# похожести слов 
model.wv.similarity('тварь', 'право')

-0.037934195

In [36]:
# самые похожие
model.wv.most_similar('топор')

[('выехать', 0.3434045910835266),
 ('вицмундир', 0.3418233096599579),
 ('нравиться', 0.3186732232570648),
 ('капернаум', 0.3163142800331116),
 ('преступник', 0.3118082284927368),
 ('случайность', 0.29301321506500244),
 ('поворотить', 0.28344807028770447),
 ('обыкновение', 0.2801940143108368),
 ('буля', 0.27475354075431824),
 ('поручить', 0.2660590708255768)]

In [37]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

[('съехать', 0.3254741132259369),
 ('раздражать', 0.31844109296798706),
 ('линия', 0.31684333086013794),
 ('покоситься', 0.30329668521881104),
 ('детски', 0.29883095622062683),
 ('чашка', 0.29052457213401794),
 ('вырываться', 0.2874678671360016),
 ('влево', 0.2853025794029236),
 ('вызывать', 0.2818695902824402),
 ('холодно', 0.2812053859233856)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [38]:
model_path = "./our_w2v.model"
model.save(model_path)

In [39]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [40]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [41]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]

In [42]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [43]:
len(sents_tokenize2)

254

Дополняем модель.

In [44]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили модель
our_model.train(sents_tokenize2, 
                total_examples=our_model.corpus_count, 
                epochs=100)

(0, 0)

In [45]:
'ядро' in model.wv.key_to_index

False

In [46]:
'ядро' in our_model.wv.key_to_index

True

In [47]:
our_model.wv.most_similar('ядро')

[('право', 0.3656421899795532),
 ('первый', 0.3411722481250763),
 ('изнеможение', 0.33265551924705505),
 ('рассеянность', 0.3231241703033447),
 ('механически', 0.3048047423362732),
 ('зе', 0.2968287169933319),
 ('далеко', 0.2877438962459564),
 ('опыт', 0.2864181101322174),
 ('выхлопотать', 0.2828163504600525),
 ('голубчик', 0.2819357216358185)]

Пример со старым словом.

In [48]:
our_model.wv.most_similar('сын')

[('помолчать', 0.33074870705604553),
 ('набережный', 0.32425230741500854),
 ('лесть', 0.3182099759578705),
 ('нарушить', 0.3141959011554718),
 ('нетерпеливо', 0.31125137209892273),
 ('гоголь', 0.30818116664886475),
 ('кончиться', 0.30746209621429443),
 ('сесть', 0.3021995425224304),
 ('снисходительный', 0.300760954618454),
 ('интересоваться', 0.29836297035217285)]

In [49]:
model.wv.most_similar('сын')

[('помолчать', 0.33074870705604553),
 ('набережный', 0.32425230741500854),
 ('лесть', 0.3182099759578705),
 ('нарушить', 0.3141959011554718),
 ('нетерпеливо', 0.31125137209892273),
 ('гоголь', 0.30818116664886475),
 ('кончиться', 0.30746209621429443),
 ('сесть', 0.3021995425224304),
 ('снисходительный', 0.300760954618454),
 ('интересоваться', 0.29836297035217285)]